For image classification:
- Binary. Activation function --> Sigmoid
- Multiclass. Activation function --> Softmax

CNN Model
- Transform images in dataset into tensors
- Pass tensors to convolutional layer
- Network generates feature maps
- Apply a non-linear activation function (i.e: RelU)
- Use pooling layer to reduce size and computational workload
- Flatten multidimensional tensors into a 1D vector
- Pass it to the fully connected layer

In [ ]:
# #to see datasets loaded
# train_dataset
# train_dataset.classes
# train_dataset.len()

## Binary vs Multiclass

In [ ]:
# Create a convolutional layer with 3 channels, 16 output channels, kernel size of 3, stride of 1, and padding of 1.
# Create a fully connected layer with an input size of 16x32x32 and a number of classes equal to 1; include only the values in the provided order (16*32*32, 1).
# Create a sigmoid activation function.

class BinaryImageClassifier(nn.Module):
    def __init__(self):
        super(BinaryImageClassifier, self).__init__()
        
        # Create a convolutional layer
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        
        # Create a fully connected layer
        self.fc = nn.Linear(16*32*32, 1)
        
        # Create an activation function
        self.sigmoid = nn.Sigmoid()

In [ ]:
# Define the __init__ method including self and num_classes as parameters.
# Create a fully connected layer with the input size of 16*32*32 and the number of classes num_classes as output.
# Create an activation function softmax with dim=1.


class MultiClassImageClassifier(nn.Module):
  
    # Define the init method
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        # Create a fully connected layer
        self.fc = nn.Linear(16*32*32, num_classes)
        
        # Create an activation function
        self.softmax = nn.Softmax(dim=1)

## Convolutional Layers for Images

Conv2d
- 1st parameter: Image color channels
    -Grasycale=1
    -RGB=3
    -Transparent=4 (alpha)


In [3]:
# To check no channels use
from torchvision.transforms import functional as F
import PIL.Image
image = PIL.Image.open('speed-limit-gantry.jpg')
num_channels = F.get_image_num_channels(image)
print("Number of channels: ", num_channels)

Number of channels:  3


### Add a conv layer

In [5]:
import torch
import torch.nn as nn

In [ ]:
#model has previously been defined
model = CNNModel()

In [6]:
# Instantiate a model from the CNNModel class and access the convolutional layers.

print("Original model: ", model)

# Create a new convolutional layer
conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)

# Append the new layer to the model
model.add_module("conv2", conv2)
print("Extended model: ", model)

NameError: name 'CNNModel' is not defined

### Creating a sequential block

In [ ]:
# In the __init__() method, define a block of convolutional layers and assign it to self.conv_block.
# In the forward() pass, pass the inputs through the convolutional block you defined.

class BinaryImageClassification(nn.Module):
  def __init__(self):
    super(BinaryImageClassification, self).__init__()
    # Create a convolutional block
    self.conv_block =  nn.Sequential(
      nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
    )
    
  def forward(self, x):
    # Pass inputs through the convolutional block
    x = self.conv_block(x)
    return x

In [ ]:
#create a new model
 

### Working with pretrained models

In [ ]:
# #saving
# torch.save(model.state_dict(), 'model_weights.pth')

# #loading
# model.load_state_dict(torch.load('model_weights.pth'))

In [ ]:
# # Save the model
# torch.save(model.state_dict(), "ModelCNN.pth")

# # Create a new model
# loaded_model = ManufacturingCNN()

# # Load the saved model
# loaded_model.load_state_dict(torch.load('ModelCNN.pth'))
# print(loaded_model)

In [14]:
#loading resnet
from torchvision.models import (
    resnet18, ResNet18_Weights
)

weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
transform = weights.transforms()

In [5]:
from PIL import Image
image = Image.open('speed-limit-gantry.jpg')
image_tensor = transforms(image)
image_reshaped = image_tensor.unsqueeze(0) #extra dimension for batch required by pytorch


In [18]:
import torch
import torch.nn as nn
#import pred
import torch.nn.functional as F
import torchvision.transforms as transforms



In [16]:
# inference

model.eval() #prepares model for inference


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
with torch.no_grad(): #disables gradient calculation
    predictions = model(image_reshaped).squeeze(0) #squeeze removes batch dimension

pred_cls = predictions.softmax(0) #converts to probabilities
cls_id = pred_cls.argmax().item() #gets the class id
cls_name = weights.meta["categories"][cls_id] #gets the class name

print(cls_name)

trailer truck
